# Fintopy #
## Pandas extensions for financial markets ##

In [1]:
import pandas as pd
from xbbg import blp
import fintopy

## Data ##

In [2]:
s = blp.bdh('MSFT US Equity', 'PX_LAST', '2021-01-01', '2021-01-31')
s.columns = s.columns.droplevel(1)
s = s.iloc[:, 0]
s.head()

2021-01-04    217.1898
2021-01-05    217.3993
2021-01-06    211.7623
2021-01-07    217.7884
2021-01-08    219.1153
Name: MSFT US Equity, dtype: float64

## Prices ##

### Set_frequency() ###

In [3]:
# Set the frequency of the series to Business Day
s.prices.set_frequency()

2021-01-04    217.1898
2021-01-05    217.3993
2021-01-06    211.7623
2021-01-07    217.7884
2021-01-08    219.1153
2021-01-11    216.9902
2021-01-12    214.4361
2021-01-13    215.8429
2021-01-14    212.5305
2021-01-15    212.1614
2021-01-18    212.1614
2021-01-19    215.9426
2021-01-20    223.8245
2021-01-21    224.4530
2021-01-22    225.4308
2021-01-25    229.0026
2021-01-26    231.7961
2021-01-27    232.3648
2021-01-28    238.3810
2021-01-29    231.4270
Freq: B, Name: MSFT US Equity, dtype: float64

In [6]:
s.prices.set_frequency('BW')

2021-01-08    219.1153
2021-01-15    212.1614
2021-01-22    225.4308
2021-01-29    231.4270
Freq: W-FRI, Name: MSFT US Equity, dtype: float64

### Rebase() ###

In [7]:
s.prices.rebase(base=200).head()

2021-01-04    200.000000
2021-01-05    200.192919
2021-01-06    195.002067
2021-01-07    200.551223
2021-01-08    201.773104
Name: MSFT US Equity, dtype: float64